In [1]:
import numpy as np
import pandas as pd
from pycirclize import Circos

In [2]:
df = pd.read_excel('../data/br2f_topographic_data.xlsx', header=0, index_col=0)

In [3]:
# define left hemisphere
brain_regions = df.index.to_numpy()

left_hemisphere = brain_regions[:392]
df_left_hemisphere = df[left_hemisphere]
df_left_hemisphere = df_left_hemisphere.head(392)

## Hippocampal - Hypothalamus connectivity plots

Next we will look at four regions (AHA, LHA, DHA, and mammillary) and their connectivity with the hippocampus. Each of the four regions is an aggregation across other subregions, so we start by summing across the subregions.

In [44]:
# construct regions
AHA_regions = np.array(['AHA', 'AHNa', 'AHNc', 'AHNp', 'AHNd'])
DHA_regions = np.array(['DMHa','DMHp','DMHv'])
mammillary_regions = np.array(['TM','MM','LM','SUMm','SUMl'])
LHA_regions = np.array([
    'LHAa','LHAjvv','LHAjvd','LHAjp','LHAjd','LHAsfa','LHAsfp','LHAs','LHAv','LHAd','LHAp'
])

hippocampus_regions = np.array(['DG','CA3','CA2','CA1v','CA1d','SUBv','SUBd','PAR', 'PRE', 'POST'])

hypothalamus_regions = np.concatenate(
    (AHA_regions, DHA_regions, mammillary_regions, LHA_regions), axis=0
)

hyp_hipp_regions = np.concatenate((hypothalamus_regions, hippocampus_regions), axis=0)

In [45]:
# reduce the left hemisphere dataframe to only include hypothalamus and hippocampal regions
df_hyp_hipp = df_left_hemisphere[hyp_hipp_regions]
df_hyp_hipp = df_hyp_hipp[df_hyp_hipp.index.isin(hyp_hipp_regions)]

In [46]:
def consolidate_sum(dataframe, new_col_row_name, subregion_array):
    ret_df = dataframe
    row_sum = ret_df[ret_df.index.isin(subregion_array)].sum(axis=0)
    ret_df.loc[new_col_row_name] = row_sum
    
    # delete extra subregions from rows
    ret_df = ret_df.loc[
    ~ret_df.index.isin(subregion_array)
    ]
    
    # columns
    col_sum = ret_df[subregion_array].sum(axis=1)
    ret_df.loc[:,new_col_row_name] = col_sum
    
    # delete extra subregion from columns
    ret_df = ret_df.drop(subregion_array, axis=1)
    return ret_df

def consolidate_average(dataframe, new_col_row_name, subregion_array):
    ret_df = dataframe
    row_sum = ret_df[ret_df.index.isin(subregion_array)].mean(axis=0)
    ret_df.loc[new_col_row_name] = row_sum
    
    # delete extra subregions from rows
    ret_df = ret_df.loc[
    ~ret_df.index.isin(subregion_array)
    ]
    
    # columns
    col_sum = ret_df[subregion_array].mean(axis=1)
    ret_df.loc[:,new_col_row_name] = col_sum
    
    # delete extra subregion from columns
    ret_df = ret_df.drop(subregion_array, axis=1)
    return ret_df

In [47]:
df_hyp_hipp = consolidate_average(df_hyp_hipp, "LHA", LHA_regions)
df_hyp_hipp = consolidate_average(df_hyp_hipp, "DHA", DHA_regions)
df_hyp_hipp = consolidate_average(df_hyp_hipp, "mammillary", mammillary_regions)

/var/folders/4k/31jdbs595pn4tslnsm4nxmzm0000gq/T/ipykernel_56341/3939621999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret_df.loc[:,new_col_row_name] = col_sum
/var/folders/4k/31jdbs595pn4tslnsm4nxmzm0000gq/T/ipykernel_56341/3939621999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret_df.loc[:,new_col_row_name] = col_sum
/var/folders/4k/31jdbs595pn4tslnsm4nxmzm0000gq/T/ipykernel_56341/3939621999.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [48]:
# NOTE - CHANGE THE AGGREGATION FUNCTION HERE TO MATCH 
# THE VERSION OF CONSOLIDATION FUNCTION USED

# AHA - custom because of naming problems
AHA_row_sum = df_hyp_hipp[df_hyp_hipp.index.isin(AHA_regions)].mean(axis=0)
df_hyp_hipp.loc["AHA_1"] = AHA_row_sum

# delete extra DHA subregions from rows
df_hyp_hipp = df_hyp_hipp.loc[
~df_hyp_hipp.index.isin(AHA_regions)
]

# columns
AHA_col_sum = df_hyp_hipp[AHA_regions].mean(axis=1)
df_hyp_hipp.loc[:,"AHA_1"] = AHA_col_sum

# delete extra LHA subregion from columns
df_hyp_hipp = df_hyp_hipp.drop(AHA_regions, axis=1)

In [49]:
# rename 'AHA_1' to 'AHA'
df_hyp_hipp = df_hyp_hipp.rename(index={"AHA_1":"AHA"})
df_hyp_hipp = df_hyp_hipp.rename(columns={"AHA_1":"AHA"})

In [50]:
df_hyp_hipp

,DG,CA3,CA2,CA1v,CA1d,SUBv,SUBd,PAR,PRE,POST,LHA,DHA,mammillary,AHA
Abbr.,,,,,,,,,,,,,,
DG,0.0,6.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00
CA3,3.0,0.000000,6.000000,6.000000,6.0,3.000000,3.0,0.0,0.0,0.0,0.181818,0.000000,0.000000,0.00
CA2,4.0,4.000000,0.000000,4.000000,0.0,3.000000,0.0,2.0,0.0,0.0,0.000000,0.000000,0.000000,0.00
CA1v,2.0,1.000000,2.000000,0.000000,0.0,7.000000,0.0,4.0,2.0,0.0,2.454545,0.333333,0.800000,1.40
CA1d,2.0,0.000000,0.000000,0.000000,0.0,0.000000,7.0,1.0,2.0,0.0,1.000000,0.000000,0.000000,0.00
SUBv,3.0,3.000000,3.000000,3.000000,0.0,0.000000,0.0,5.0,4.0,0.0,2.818182,3.333333,1.800000,2.60
SUBd,2.0,3.000000,0.000000,0.000000,4.0,0.000000,0.0,0.0,0.0,4.0,2.000000,0.000000,1.600000,0.20
PRE,2.0,2.000000,2.000000,2.000000,2.0,4.000000,4.0,4.0,0.0,4.0,0.181818,0.000000,0.800000,0.00
POST,2.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,4.0,5.0,0.0,0.090909,0.000000,0.800000,0.00


In [51]:
# remove self-connections (set diagonal to 0)
for i in range(len(df_hyp_hipp.index)):
    df_hyp_hipp.iloc[i,i] = 0

In [52]:
# we need to remove connections within the hypothalamus--
# we're only interested in hippocampal-hypothalamus connectivity

hypothalamus_aggregated_regions = np.array(["LHA",'DHA', 'mammillary', 'AHA'])
for subregion_i in range(len(hypothalamus_aggregated_regions)):
    for subregion_j in range(len(hypothalamus_aggregated_regions)):
        df_hyp_hipp.loc[
        hypothalamus_aggregated_regions[subregion_i], 
        hypothalamus_aggregated_regions[subregion_j]
        ] = 0

In [53]:
df_hyp_hipp

,DG,CA3,CA2,CA1v,CA1d,SUBv,SUBd,PAR,PRE,POST,LHA,DHA,mammillary,AHA
Abbr.,,,,,,,,,,,,,,
DG,0.0,6.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
CA3,3.0,0.000000,6.000000,6.000000,6.0,3.000000,3.0,0.0,0.0,0.0,0.181818,0.000000,0.0,0.0
CA2,4.0,4.000000,0.000000,4.000000,0.0,3.000000,0.0,2.0,0.0,0.0,0.000000,0.000000,0.0,0.0
CA1v,2.0,1.000000,2.000000,0.000000,0.0,7.000000,0.0,4.0,2.0,0.0,2.454545,0.333333,0.8,1.4
CA1d,2.0,0.000000,0.000000,0.000000,0.0,0.000000,7.0,1.0,2.0,0.0,1.000000,0.000000,0.0,0.0
SUBv,3.0,3.000000,3.000000,3.000000,0.0,0.000000,0.0,5.0,4.0,0.0,2.818182,3.333333,1.8,2.6
SUBd,2.0,3.000000,0.000000,0.000000,4.0,0.000000,0.0,0.0,0.0,4.0,2.000000,0.000000,1.6,0.2
PRE,2.0,2.000000,2.000000,2.000000,2.0,4.000000,4.0,0.0,0.0,4.0,0.181818,0.000000,0.8,0.0
POST,2.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,4.0,0.0,0.0,0.090909,0.000000,0.8,0.0


In [54]:
#clear out intrinsic connections between hippocampus_regions
for subregion_i in range(len(hippocampus_regions)):
    for subregion_j in range(len(hippocampus_regions)):
        df_hyp_hipp.loc[
        hippocampus_regions[subregion_i], 
        hippocampus_regions[subregion_j]
        ] = 0

In [55]:
df_hyp_hipp

,DG,CA3,CA2,CA1v,CA1d,SUBv,SUBd,PAR,PRE,POST,LHA,DHA,mammillary,AHA
Abbr.,,,,,,,,,,,,,,
DG,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
CA3,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.181818,0.000000,0.0,0.0
CA2,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
CA1v,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,2.454545,0.333333,0.8,1.4
CA1d,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.0
SUBv,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,2.818182,3.333333,1.8,2.6
SUBd,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,2.000000,0.000000,1.6,0.2
PRE,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.181818,0.000000,0.8,0.0
POST,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.090909,0.000000,0.8,0.0


In [57]:
circos_hyp_hipp_extrinsic_only = Circos.initialize_from_matrix(df_hyp_hipp, space=5, cmap="jet")

circos_hyp_hipp_extrinsic_only.savefig("../output/hippocampus_hypothalamus_extrinsic_average_connectivity.svg")


